# Deploy your final model
* To succecfully run this notebook you need a python3.7 kernel with requirements in ./sagemaker-custom-image/requirements.txt

# This notebook will help you to do:
* Pre-process raw data according to model requirements
* Test prediction with the final model
* Post-process the predict data and create the rating variable
* Create the script handler.py for deploy using serverless

# 1. Parameters

In [1]:
#Name
ModelName = 'prop_apply_prospects'
#Version
ModelVersion = 'v12'
#Model ID
ModelId = ModelName+'_'+ModelVersion

#Setting the model target variable name
VarTarget = 'target'

#process outputs such as MOJO model, images and performance of tested models
PathModelMojo = './output_model/models/best/GBM_grid_1_AutoML_1_20220628_13109_model_7.zip/'

#If you have a huge dataset, I should consider use a small sample for first execution
PctSampleSize = 1

#Bucket already created on S3
bucket = 'data-science-lab'

#Selected Feature difined in Data_Prep
CAT = [ 'declared_seniority'
       ,'last_company_classification'
       ,'company_classification_migration'
       ,'job_area'
       ,'declared_seniority_migration'
       ,'prospect_location_state'
       ,'prospect_area_migration'
      ]
#float
NUM = [ 'prospect_smart_skills_qty'
       ,'prospect_experiences_qty'
       ,'prospect_companies_qty'
       ,'total_experience_months'
       ,'experience_duration_months_min'
       ,'experience_duration_months_clean_avg'
       ,'experience_duration_months_clean_stddev'
       ,'max_salary_offered'
       ,'last_experience_duration_months_to_avg'
       ,'prospect_linkedin_about_word_count'
       ,'last_experience_descriptions_word_count'       
       ,'import_policy_word_count'
       ,'job_technical_requirements_word_count'
       ,'job_validation_questions_word_count'
       ,'all_company_classifications_count'
      ]
selected_features = CAT + NUM

# 2. Import Libraries

In [3]:
import sys
sys.path.append('/var/lang/lib/python37.zip')
sys.path.append('/var/lang/lib/python3.7')
sys.path.append('/var/lang/lib/python3.7/lib-dynload')
sys.path.append('/var/lang/lib/python3.7/site-packages')
sys.path.remove('/opt/.sagemakerinternal/conda/lib/python3.7/site-packages')

In [4]:
import h2o
import pandas as pd
import numpy as np
import json
import boto3
import queue
from threading import Thread
from boto3.dynamodb.conditions import Key
from datetime import datetime as dt

# 3. Read raw data from application source and transform before predict

In [5]:
#It is necessary to create two boto3 to work with multiple threads. One for each thread

#boto3_Session para a tabela jobs
boto_jobs = boto3.Session(region_name='us-east-1',
    aws_access_key_id='xxxx',
    aws_secret_access_key='xxxxx',
    aws_session_token='xxxxxx')

#boto3_Session para a tabela prospects
boto_prospects = boto3.Session(region_name='us-east-1',
    aws_access_key_id='xxxx',
    aws_secret_access_key='xxxxx',
    aws_session_token='xxxxxx')

In [7]:
#Sample raw data from application
event={
    "queryStringParameters":{"jobId":"2300",
                             "linkedinUsername":"fernando-heitor"}
}

In [9]:
linkedinUsername = event['queryStringParameters']['linkedinUsername']
jobId = event['queryStringParameters']['jobId']

In [10]:
#Criar funcoes de consulta ao DynamoDB para serem executadas em paralelo diminuindo o tempo
def dynamodb_jobs_clients_companies(jobId, boto3):
    jobsTable = boto3.resource('dynamodb').Table('prod-jobsTable').query(
        IndexName='GSI1',
        KeyConditionExpression=Key('backofficeId').eq(int(jobId))
    )['Items']

    clientsTable = boto3.resource('dynamodb').Table('prod-clientsTable').query(
        IndexName='idBackofficeIndex',
        KeyConditionExpression = (Key('source').eq('backoffice') & Key('idBackoffice').eq(int(jobsTable[0]['companyId'])))
    )['Items']
    
    if clientsTable[0]['linkedinURL'][-1] == '/':
        linkedin_company_user_name = clientsTable[0]['linkedinURL'].split('/')[-2]
    else:
        linkedin_company_user_name = clientsTable[0]['linkedinURL'].split('/')[-1]

    linkedinCompanies = boto3.resource('dynamodb').Table('prod-linkedinCompanies').query(
        IndexName='LinkedinUsernameIndex',
        KeyConditionExpression=Key('linkedinUsername').eq(linkedin_company_user_name)
    )['Items']
    
    return_object = {} 
    return_object['jobsTable'] = jobsTable
    return_object['linkedinCompanies'] = linkedinCompanies
    return return_object

In [11]:
def company_classification_const(s):
    if s == 'Empresa pequena':
        out='Empresa pequena'
    elif s == 'Empresas pequenas':
        out='Empresa pequena'
    elif s == 'Empresa Tradicional A':
        out='Empresa Tradicional A'
    elif s == 'Empresa Tradicional B':
        out='Empresa Tradicional B'
    elif s == 'Empresa Tradicional C':
        out='Empresa Tradicional C'
    elif s == 'High Tech A':
        out='High Tech A'
    elif s == 'High Tech B':
        out='High Tech B'
    elif s == 'High Tech C':
        out='High Tech C'
    elif s == 'High Tech D':
        out='High Tech D'
    elif s == 'Software House A':
        out='Software House A'
    elif s == 'Software House B':
        out='Software House B'
    elif s == 'Software House C':
        out='Software House C'
    elif s == 'not_mapped':
        out='Missing'
    elif s in ('', 'null', None, 'None'):
        out='Missing'
    else:
        out='UNKNOWN'
    return out

def dynamodb_prospects_companies(linkedinUsername, boto3):
    prospectsTable = boto3.resource('dynamodb').Table('prod-prospectsTable').get_item(Key={'linkedinUsername': linkedinUsername})['Item']

    dict_prospects={}
    total_experience_months=[]
    total_experience_months_clean=[]
    prospect_companies=[]
    all_company_classifications=[]
    
    for i in range(len(prospectsTable['experiences'])):
        try:        
            prospect_companies.append(prospectsTable['experiences'][i]['linkedinCompanyName'])

            try:
                linkedinCompanies = boto3.resource('dynamodb').Table('prod-linkedinCompanies').query(
                    IndexName='LinkedinUsernameIndex',
                    KeyConditionExpression=Key('linkedinUsername').eq(prospectsTable['experiences'][i]['linkedinCompanyUsername'])
                )['Items']
                all_company_classifications.append(company_classification_const(linkedinCompanies[0]['category'].replace(' - ', ' ')))
            except:
                try:
                    linkedinCompanies = boto3.resource('dynamodb').Table('prod-linkedinCompanies').query(
                        IndexName='LinkedinIdIndex',
                        KeyConditionExpression=Key('linkedinId').eq(int(prospectsTable['experiences'][i]['linkedinCompanyId']))
                    )['Items']
                    all_company_classifications.append(company_classification_const(linkedinCompanies[0]['category'].replace(' - ', ' ')))
                except:
                    all_company_classifications.append('Missing')

            if i == 0:
                if prospectsTable['experiences'][i]['to'] == None:
                    total_experience_months.append((int((dt.now() - dt.strptime(prospectsTable['experiences'][i]['from'][:10], "%Y-%m-%d")).days/30)))
                else:
                    total_experience_months.append((int((dt.strptime(prospectsTable['experiences'][i]['to'][:10], "%Y-%m-%d") - dt.strptime(prospectsTable['experiences'][i]['from'][:10], "%Y-%m-%d")).days/30)))
                #precisa colocar o copy porque a variavel total_experience sera alterada, mas a last_experience nao pode mais ser alterada
                last_experience_duration_months = total_experience_months.copy()
                try:
                    last_experience_descriptions = prospectsTable['experiences'][i]['description']
                except:
                    last_experience_descriptions = ''            
            else:
                if prospectsTable['experiences'][i]['to'] == None:
                    total_experience_months.append((int((dt.now() - dt.strptime(prospectsTable['experiences'][i]['from'][:10], "%Y-%m-%d")).days/30)))
                    total_experience_months_clean.append((int((dt.now() - dt.strptime(prospectsTable['experiences'][i]['from'][:10], "%Y-%m-%d")).days/30)))
                else:
                    total_experience_months.append((int((dt.strptime(prospectsTable['experiences'][i]['to'][:10], "%Y-%m-%d") - dt.strptime(prospectsTable['experiences'][i]['from'][:10], "%Y-%m-%d")).days/30)))
                    total_experience_months_clean.append((int((dt.strptime(prospectsTable['experiences'][i]['to'][:10], "%Y-%m-%d") - dt.strptime(prospectsTable['experiences'][i]['from'][:10], "%Y-%m-%d")).days/30)))
        except:
            last_experience_duration_months=[0]
            total_experience_months=[0]
            last_experience_descriptions = ''
            all_company_classifications = ['Missing']

    dict_prospects['prospect_companies_qty'] = len(set(prospect_companies))
    dict_prospects['total_experience_months'] = sum(total_experience_months)
    dict_prospects['experience_duration_months_min'] = min(total_experience_months)
    dict_prospects['experience_duration_months_clean_avg'] = np.mean((total_experience_months_clean))
    dict_prospects['experience_duration_months_clean_stddev'] =  np.std(total_experience_months_clean, ddof = 1)
    dict_prospects['last_experience_duration_months_to_avg'] = float(last_experience_duration_months[0]) - float(dict_prospects['experience_duration_months_clean_avg'])
    dict_prospects['last_experience_descriptions_word_count'] = float(len(last_experience_descriptions.split()))
    dict_prospects['last_company_classification'] = all_company_classifications[0]
    dict_prospects['all_company_classifications_word_count'] = float(len('; '.join(map(str, all_company_classifications)).replace("||", ",").split()))
    
    return_object = {} 
    return_object['prospectsTable'] = prospectsTable
    return_object['dict_prospects'] = dict_prospects
    return return_object

In [12]:
# dynamodb_jobs_clients_companies(jobId, boto3)

In [13]:
#criar q1 e q2 para receber o return de cada funcao
q1 = queue.Queue()
q2 = queue.Queue()
#criar as duas tarefas
t1 = Thread(target = lambda q, arg1, arg2 : q.put(dynamodb_jobs_clients_companies(arg1, arg2)), args = (q1, jobId, boto_jobs))
t2 = Thread(target = lambda q, arg1, arg2 : q.put(dynamodb_prospects_companies(arg1, arg2)), args = (q2, linkedinUsername, boto_prospects))
#iniciar as duas tarefas simutaneamente.
t1.start()
t2.start()
#unir as duas tarefas para garantir que a proxima tarefa seja iniciada somente quando as duas terminarem
t1.join()
t2.join()

#recebar o return da funcao
while not q1.empty():
    return_object_jobs_clients_companies = q1.get()
while not q2.empty():
    return_object_prospects_companies = q2.get()

In [14]:
jobsTable = return_object_jobs_clients_companies['jobsTable']
linkedinCompanies = return_object_jobs_clients_companies['linkedinCompanies']
prospectsTable = return_object_prospects_companies['prospectsTable']
dict_prospects = return_object_prospects_companies['dict_prospects']

In [15]:
dict_prospects

{'prospect_companies_qty': 5,
 'total_experience_months': 80,
 'experience_duration_months_min': 3,
 'experience_duration_months_clean_avg': 15.4,
 'experience_duration_months_clean_stddev': 12.973048986263793,
 'last_experience_duration_months_to_avg': -12.4,
 'last_experience_descriptions_word_count': 8.0,
 'last_company_classification': 'Missing',
 'all_company_classifications_word_count': 12.0}

## prod-jobsTable

In [16]:
# declared_seniority_migration
# prospect_area_migration
# max_salary_offered
# import_policy_word_count
# job_technical_requirements_word_count
# job_validation_questions_word_count
# company_classification_migration

In [17]:
#original columns names in JobsTable in DynamoDB
declared_seniority_migration = job_seniority = 'profile.seniority'
prospect_area_migration = job_area = 'area'
max_salary_offered = 'maxsalary'
import_policy_word_count = 'importPolicy'
job_technical_requirements_word_count = 'alignments.intendedTechnicalInfo'
job_validation_questions_word_count = 'validationquestions'
company_classification_migration = company_classification = "last_company_classification"

In [18]:
# jobsTable = boto3.resource('dynamodb').Table('prod-jobsTable').query(
#     IndexName='GSI1',
#     KeyConditionExpression=Key('backofficeId').eq(int(event['queryStringParameters']['jobId']))
# )['Items']
# jobsTable

In [19]:
#Start empty dictionary
dict_jobs={}

In [20]:
## job_seniority = 'profile.seniority'
def job_seniority_const(s):
    if s in ("Júnior", "junior", "Junior"):
        out="Junior"
    elif s in ("Pleno", "pleno"):
        out="Mid-level"
    elif s in ("Senior", "Sênior", "senior"):
        out="Senior"
    elif s in ("Especialista", "especialista"):
        out="Specialist"
    elif s == "Tech Lead":
        out="Tech Lead"
    elif s == "Tech Manager":
        out="Tech Manager"
    elif s in ('', 'null', None, 'None'):
        out="Missing"
    else:
        out="UNKNOWN"
    return out

dict_jobs['job_seniority'] = job_seniority_const(jobsTable[0]['profile']['seniority'])
dict_jobs

{'job_seniority': 'Specialist'}

In [21]:
## job_area = 'area'
try:
    dict_jobs['job_area'] = jobsTable[0]['area']
except:
    dict_jobs['job_area'] = 'Missing'
dict_jobs

{'job_seniority': 'Specialist', 'job_area': 'Agile'}

In [22]:
## max_salary_offered = 'maxsalary'
try:
    dict_jobs['max_salary_offered'] = float(jobsTable[0]['maxSalary'])
except:
    dict_jobs['max_salary_offered'] = 0
dict_jobs

{'job_seniority': 'Specialist',
 'job_area': 'Agile',
 'max_salary_offered': 14000.0}

In [23]:
## import_policy_word_count = 'importPolicy'
try:
    dict_jobs['import_policy_word_count'] = float(len(jobsTable[0]['importPolicy'].split()))
except:
    dict_jobs['import_policy_word_count'] = 0
dict_jobs

{'job_seniority': 'Specialist',
 'job_area': 'Agile',
 'max_salary_offered': 14000.0,
 'import_policy_word_count': 21.0}

In [24]:
## job_technical_requirements_word_count = 'alignments.intendedTechnicalInfo'
try:
    dict_jobs['job_technical_requirements_word_count'] = float(len(jobsTable[0]['alignments']['intendedTechnicalInfo'].split()))
except:
    dict_jobs['job_technical_requirements_word_count'] = 0
dict_jobs

{'job_seniority': 'Specialist',
 'job_area': 'Agile',
 'max_salary_offered': 14000.0,
 'import_policy_word_count': 21.0,
 'job_technical_requirements_word_count': 135.0}

In [25]:
## job_validation_questions_word_count = 'validationQuestions'
try:
    dict_jobs['job_validation_questions_word_count'] = float(len(jobsTable[0]['validationQuestions'].split()))
except:
    dict_jobs['job_validation_questions_word_count'] = 0
dict_jobs

{'job_seniority': 'Specialist',
 'job_area': 'Agile',
 'max_salary_offered': 14000.0,
 'import_policy_word_count': 21.0,
 'job_technical_requirements_word_count': 135.0,
 'job_validation_questions_word_count': 126.0}

In [26]:
#company_classification
try:
#     clientsTable = boto3.resource('dynamodb').Table('prod-clientsTable').query(
#         IndexName='idBackofficeIndex',
#         KeyConditionExpression = (Key('source').eq('backoffice') & Key('idBackoffice').eq(int(jobsTable[0]['companyId'])))
#     )['Items']

#     if clientsTable[0]['linkedinURL'][-1] == '/':
#         linkedin_company_user_name = clientsTable[0]['linkedinURL'].split('/')[-2]
#     else:
#         linkedin_company_user_name = clientsTable[0]['linkedinURL'].split('/')[-1]
    
#     linkedinCompanies = boto3.resource('dynamodb').Table('prod-linkedinCompanies').query(
#         IndexName='LinkedinUsernameIndex',
#         KeyConditionExpression=Key('linkedinUsername').eq(linkedin_company_user_name)
#     )['Items']

    dict_jobs['job_company_classification'] = company_classification_const(linkedinCompanies[0]['category'].replace(' - ', ' '))
except:
    dict_jobs['job_company_classification'] = 'Missing'
    
dict_jobs

{'job_seniority': 'Specialist',
 'job_area': 'Agile',
 'max_salary_offered': 14000.0,
 'import_policy_word_count': 21.0,
 'job_technical_requirements_word_count': 135.0,
 'job_validation_questions_word_count': 126.0,
 'job_company_classification': 'Empresa Tradicional B'}

## prod-prospectsTable

In [27]:
# 'declared_seniority' ok
# 'last_company_classification' ok
# 'company_classification_migration' ok
# 'declared_seniority_migration' ok
# 'prospect_location_state' ok
# 'prospect_area_migration' ok
# 'prospect_smart_skills_qty' ok
# 'prospect_experiences_qty' ok
# 'prospect_companies_qty' ok
# 'total_experience_months' ok 
# 'experience_duration_months_min' ok
# 'experience_duration_months_clean_avg' ok
# 'experience_duration_months_clean_stddev' ok
# 'last_experience_duration_months_to_avg' ok
# 'prospect_linkedin_about_word_count' ok
# 'last_experience_descriptions_word_count' ok
# 'all_company_classifications_word_count' 
#Colocar try exepction em todas

In [28]:
#original columns names in ProspectsTable in DynamoDB
declared_seniority = 'declaredseniority'

prospect_smart_skills_qty ="""
#Treat the stack column
prospects_smart_skills1 = sdf_prospects_dedup4.select("linkedin_user_name", F.explode("smarttags"))

#Rename columns
prospects_smart_skills2 = prospects_smart_skills1.withColumn('prospect_smart_skill', F.col('col.tag'))\
.withColumn('smart_skill_count', F.col('col.count')).drop('col')

#handle nomenclature of skills
prospects_smart_skills3 = prospects_smart_skills2.withColumn('prospect_smart_skill', F.translate(F.lower(F.col('prospect_smart_skill')), " ", "-"))\
.withColumn('prospect_smart_skill', F.translate(F.col('prospect_smart_skill'), "/", "-"))\
.withColumn('prospect_smart_skill', F.translate(F.col('prospect_smart_skill'), ".", ""))

select "linkedin_user_name", count("prospect_smart_skill") as prospect_smart_skills_qty, sum("smart_skill_count") as prospect_smart_skills_sum, avg("smart_skill_count") as prospect_smart_skills_avg ,array_agg(prospect_smart_skill) as all_prospect_smart_skills from "prod-lakehouse-mirror"."prospects_smart_skills" group by "linkedin_user_name"
"""

prospect_experiences_qty ="""
#Treat the stack column
prospects_experiences1 = sdf_prospects_dedup4.select("linkedin_user_name", F.explode("experiences"))
#Flatten nested columns
prospects_experiences2 = flatten_df(prospects_experiences1)
#Rename columns
prospects_experiences3 = prospects_experiences2.withColumnRenamed('col_linkedinCompanyName', 'linkedin_company_name')\
.withColumn('experience_position', F.coalesce(F.col('col_role'), F.col('col_position')).cast(StringType()))\
.withColumn('company_linkedin_id', F.coalesce(F.col('col_id'), F.col('col_linkedincompanyid')).cast(LongType()))\
.withColumnRenamed('col_linkedinCompanyUsername', 'linkedin_company_user_name')\
.withColumnRenamed('col_description', 'experience_description')\
.withColumn('experience_date_from', F.substring('col_from', 1, 10))\
.withColumn('experience_date_to', F.substring('col_to', 1, 10))\
.withColumn('experience_date_to_tmp', F.when(F.col('experience_date_to').isNull(), dt.datetime.now()).otherwise(F.col('experience_date_to')))\
.withColumn('experience_duration_months', ((F.datediff(F.col('experience_date_to_tmp'), F.col('experience_date_from')))/30).cast(LongType())).drop('col_to','col_from', 'experience_date_to_tmp', 'col_role', 'col_position', 'col_id', 'col_linkedincompanyid')

,count(c2.linkedin_company_name) as prospect_experiences_qty
from "prod-lakehouse-mirror"."prospects_experiences"
"""
prospect_companies_qty = ',count(distinct c2.linkedin_company_name) as prospect_companies_qty'

total_experience_months = """
#Treat the stack column
prospects_experiences1 = sdf_prospects_dedup4.select("linkedin_user_name", F.explode("experiences"))
#Flatten nested columns
prospects_experiences2 = flatten_df(prospects_experiences1)
#Rename columns
prospects_experiences3 = prospects_experiences2.withColumnRenamed('col_linkedinCompanyName', 'linkedin_company_name')\
.withColumn('experience_position', F.coalesce(F.col('col_role'), F.col('col_position')).cast(StringType()))\
.withColumn('company_linkedin_id', F.coalesce(F.col('col_id'), F.col('col_linkedincompanyid')).cast(LongType()))\
.withColumnRenamed('col_linkedinCompanyUsername', 'linkedin_company_user_name')\
.withColumnRenamed('col_description', 'experience_description')\
.withColumn('experience_date_from', F.substring('col_from', 1, 10))\
.withColumn('experience_date_to', F.substring('col_to', 1, 10))\
.withColumn('experience_date_to_tmp', F.when(F.col('experience_date_to').isNull(), dt.datetime.now()).otherwise(F.col('experience_date_to')))\
.withColumn('experience_duration_months', ((F.datediff(F.col('experience_date_to_tmp'), F.col('experience_date_from')))/30).cast(LongType()))

,sum(c2.experience_duration_months) as total_experience_months'
"""

experience_duration_months_min = "min(c2.experience_duration_months) as experience_duration_months_min"

experience_duration_months_clean_avg = """
,b2.experience_duration_months as experience_duration_months_tmp1

,case when experience_rank = 1 then null else c3.experience_duration_months_tmp1 end as experience_duration_months_clean

,avg(c2.experience_duration_months_clean) as experience_duration_months_clean_avg
"""
experience_duration_months_clean_stddev = 'stddev(c2.experience_duration_months_clean) as experience_duration_months_clean_stddev'

last_company_classification = """
def company_classification_const(s):
    if s == 'Empresa pequena':
        out='Empresa pequena'
    elif s == 'Empresas pequenas':
        out='Empresa pequena'
    elif s == 'Empresa Tradicional A':
        out='Empresa Tradicional A'
    elif s == 'Empresa Tradicional B':
        out='Empresa Tradicional B'
    elif s == 'Empresa Tradicional C':
        out='Empresa Tradicional C'
    elif s == 'High Tech A':
        out='High Tech A'
    elif s == 'High Tech B':
        out='High Tech B'
    elif s == 'High Tech C':
        out='High Tech C'
    elif s == 'High Tech D':
        out='High Tech D'
    elif s == 'Software House A':
        out='Software House A'
    elif s == 'Software House B':
        out='Software House B'
    elif s == 'Software House C':
        out='Software House C'
    elif s == 'not_mapped':
        out='Missing'
    elif s in ('', 'null', None, 'None'):
        out='Missing'
    else:
        out='UNKNOWN'
    return out
company_classification_func = F.udf(company_classification_const, StringType())

sdf_linkedincompanies_dedup2 = sdf_linkedincompanies_dedup1.drop('partition_0', 'partition_1', 'partition_2', 'partition_3')\
.withColumn("createdat", (F.to_timestamp("createdat") - F.expr('INTERVAL 3 HOURS')))\
.withColumn("updatedat", (F.to_timestamp("updatedat") - F.expr('INTERVAL 3 HOURS')))\
.withColumnRenamed('searchcount', 'qty_search')\
.withColumnRenamed('linkedinusername', 'linkedin_company_user_name')\
.withColumnRenamed('linkedinid', 'company_linkedin_id')\
.withColumnRenamed('city', 'company_city')\
.withColumnRenamed('companysize', 'company_size')\
.withColumnRenamed('name', 'company_name')\
.withColumnRenamed('founded', 'company_foundation_year')\
.withColumnRenamed('employeecount', 'qty_company_employees')\
.withColumnRenamed('categorydescription', 'classification_description')\
.withColumnRenamed('state', 'company_state')\
.withColumn('category', F.regexp_replace('category', ' - ', ' '))\
.withColumn("company_classification", company_classification_func(F.trim(F.col('category'))))\

datasource_companies_id = glueContext.create_dynamic_frame.from_catalog(database = "prod-lakehouse-mirror", table_name = "companies", transformation_ctx = "datasource_companies", additional_options={"mergeSchema": "true"}).toDF()\
.select('linkedin_company_user_name', 'company_linkedin_id', 'company_classification').where(F.col('company_classification') != 'Missing')\
.withColumn('linkedin_company_user_name_tmp', F.col('linkedin_company_user_name')).dropDuplicates(subset = ["company_linkedin_id"])

prospects_experiences6 = prospects_experiences5.alias('df1').join(datasource_companies_username.alias('df2'),
                              on = prospects_experiences5['linkedin_company_user_name'] == datasource_companies_username['linkedin_company_user_name'],
                              how = 'left')\
                         .select('df1.*',
                                 'df2.company_classification').withColumnRenamed('company_classification', 'company_classification_user').alias('df3').join(datasource_companies_id.alias('df4'),
                              on = prospects_experiences5['company_linkedin_id'] == datasource_companies_id['company_linkedin_id'],
                              how = 'left')\
                         .select('df3.*',
                                 'df4.company_classification',
                                 'df4.linkedin_company_user_name_tmp').withColumnRenamed('company_classification', 'company_classification_id')

prospects_experiences7 = prospects_experiences6.withColumn('company_classification', F.coalesce(F.col('company_classification_user'), F.col('company_classification_id')).cast(StringType()))\
.withColumn('company_classification', F.coalesce(F.col('company_classification'), F.lit('Missing')).cast(StringType()))\
.withColumn('linkedin_company_user_name', F.coalesce(F.col('linkedin_company_user_name'), F.col('linkedin_company_user_name_tmp')).cast(StringType()))

first_value(b2.company_classification) over(partition by a2.linkedin_user_name, a2.approach_id order by b2.experience_date_from desc) as last_company_classification'
"""

company_classification_migration = last_company_classification

In [29]:
# prospectsTable = boto3.resource('dynamodb').Table('prod-prospectsTable').get_item(Key={'linkedinUsername': event['queryStringParameters']['linkedinUsername']})['Item']
# prospectsTable

In [30]:
#Start empty dictionary
# dict_prospects={}

In [31]:
## declared_seniority = 'declaredseniority'
try:
    dict_prospects['declared_seniority'] = job_seniority_const(prospectsTable['declaredSeniority'])
except:
    dict_prospects['declared_seniority'] = 'Missing'
dict_prospects

{'prospect_companies_qty': 5,
 'total_experience_months': 80,
 'experience_duration_months_min': 3,
 'experience_duration_months_clean_avg': 15.4,
 'experience_duration_months_clean_stddev': 12.973048986263793,
 'last_experience_duration_months_to_avg': -12.4,
 'last_experience_descriptions_word_count': 8.0,
 'last_company_classification': 'Missing',
 'all_company_classifications_word_count': 12.0,
 'declared_seniority': 'Mid-level'}

In [32]:
# 'declared_seniority_migration'
dict_prospects['declared_seniority_migration'] = (dict_prospects['declared_seniority']+'-to-'+dict_jobs['job_seniority']).strip()
dict_prospects

{'prospect_companies_qty': 5,
 'total_experience_months': 80,
 'experience_duration_months_min': 3,
 'experience_duration_months_clean_avg': 15.4,
 'experience_duration_months_clean_stddev': 12.973048986263793,
 'last_experience_duration_months_to_avg': -12.4,
 'last_experience_descriptions_word_count': 8.0,
 'last_company_classification': 'Missing',
 'all_company_classifications_word_count': 12.0,
 'declared_seniority': 'Mid-level',
 'declared_seniority_migration': 'Mid-level-to-Specialist'}

In [33]:
# prospect_location_state

#Criacao das variaveis de pais com base na variavel propesct_location do linkedin. Para comparar abordagens de talentos que moram fora do Brasil
def prospect_country(country):
    if pd.isna(country):
        return "Missing"
    elif country == 'São Paulo':
        return 'Brazil'
    elif country == 'Rio de Janeiro':
        return 'Brazil'
    elif country == 'Campinas':
        return 'Brazil'
    elif country == 'Belo Horizonte':
        return 'Brazil'
    elif country == 'Porto Alegre':
        return 'Brazil'
    elif country == 'Curitiba':
        return 'Brazil'
    elif country == 'Brasília':
        return 'Brazil'
    elif country == 'Florianópolis':
        return 'Brazil'
    elif country == 'Salvador':
        return 'Brazil'
    elif country == 'Fortaleza':
        return 'Brazil'
    elif country == 'Recife':
        return 'Brazil'
    elif country == 'Manaus':
        return 'Brazil'
    elif country == 'Ribeirão Preto':
        return 'Brazil'
    elif country == 'Goiânia':
        return 'Brazil'
    elif country == 'João Pessoa':
        return 'Brazil'
    elif country == 'Londrina':
        return 'Brazil'
    elif country == 'Vitória':
        return 'Brazil'
    elif country == 'Cuiabá':
        return 'Brazil'
    elif country == 'Greater São Paulo Area':
        return 'Brazil'
    elif country == 'Natal':
        return 'Brazil'
    elif country == 'São luis':
        return 'Brazil'
    elif country == 'Brazil':
        return 'Brazil'
    elif country == 'Brasil':
        return 'Brazil'
    else:
        return 'Others'
    
def prospect_region_international(region, country):
    if country not in ('Brazil', 'Missing'):
        return "International"
    else:
        return region
try:
    dict_prospects['prospect_location_state'] = prospect_region_international(prospectsTable['location'].split(',')[-2].replace(" e Região", "").strip(), prospect_country(prospectsTable['location'].split(',')[-1].replace(" e Região", "").strip()))
except:
    dict_prospects['prospect_location_state'] = 'Missing'
dict_prospects

{'prospect_companies_qty': 5,
 'total_experience_months': 80,
 'experience_duration_months_min': 3,
 'experience_duration_months_clean_avg': 15.4,
 'experience_duration_months_clean_stddev': 12.973048986263793,
 'last_experience_duration_months_to_avg': -12.4,
 'last_experience_descriptions_word_count': 8.0,
 'last_company_classification': 'Missing',
 'all_company_classifications_word_count': 12.0,
 'declared_seniority': 'Mid-level',
 'declared_seniority_migration': 'Mid-level-to-Specialist',
 'prospect_location_state': 'São Paulo'}

In [34]:
## prospect_area_migration
try:    
    dict_prospects['prospect_area_migration'] = (prospectsTable['area'].split()[0]+'-to-'+dict_jobs['job_area']).strip()
except:
    dict_prospects['prospect_area_migration'] = ('Missing'+'-to-'+dict_jobs['job_area']).strip()
dict_prospects

{'prospect_companies_qty': 5,
 'total_experience_months': 80,
 'experience_duration_months_min': 3,
 'experience_duration_months_clean_avg': 15.4,
 'experience_duration_months_clean_stddev': 12.973048986263793,
 'last_experience_duration_months_to_avg': -12.4,
 'last_experience_descriptions_word_count': 8.0,
 'last_company_classification': 'Missing',
 'all_company_classifications_word_count': 12.0,
 'declared_seniority': 'Mid-level',
 'declared_seniority_migration': 'Mid-level-to-Specialist',
 'prospect_location_state': 'São Paulo',
 'prospect_area_migration': 'Data-to-Agile'}

In [35]:
## prospect_smart_skills_qty
try:
    dict_prospects['prospect_smart_skills_qty'] = float(len(prospectsTable['smartTags']))
except:
    dict_prospects['prospect_smart_skills_qty'] = 0.0
dict_prospects

{'prospect_companies_qty': 5,
 'total_experience_months': 80,
 'experience_duration_months_min': 3,
 'experience_duration_months_clean_avg': 15.4,
 'experience_duration_months_clean_stddev': 12.973048986263793,
 'last_experience_duration_months_to_avg': -12.4,
 'last_experience_descriptions_word_count': 8.0,
 'last_company_classification': 'Missing',
 'all_company_classifications_word_count': 12.0,
 'declared_seniority': 'Mid-level',
 'declared_seniority_migration': 'Mid-level-to-Specialist',
 'prospect_location_state': 'São Paulo',
 'prospect_area_migration': 'Data-to-Agile',
 'prospect_smart_skills_qty': 23.0}

In [36]:
## prospect_experiences_qty
try:
    dict_prospects['prospect_experiences_qty'] = float(len(prospectsTable['experiences']))
except:
    dict_prospects['prospect_experiences_qty'] = 0.0
dict_prospects

{'prospect_companies_qty': 5,
 'total_experience_months': 80,
 'experience_duration_months_min': 3,
 'experience_duration_months_clean_avg': 15.4,
 'experience_duration_months_clean_stddev': 12.973048986263793,
 'last_experience_duration_months_to_avg': -12.4,
 'last_experience_descriptions_word_count': 8.0,
 'last_company_classification': 'Missing',
 'all_company_classifications_word_count': 12.0,
 'declared_seniority': 'Mid-level',
 'declared_seniority_migration': 'Mid-level-to-Specialist',
 'prospect_location_state': 'São Paulo',
 'prospect_area_migration': 'Data-to-Agile',
 'prospect_smart_skills_qty': 23.0,
 'prospect_experiences_qty': 6.0}

In [37]:
## prospect_experiences
# total_experience_months=[]
# total_experience_months_clean=[]
# prospect_companies=[]
# all_company_classifications=[]
# for i in range(len(prospectsTable['experiences'])):
#     try:        
#         prospect_companies.append(prospectsTable['experiences'][i]['linkedinCompanyName'])
        
#         try:
#             linkedinCompanies = boto3.resource('dynamodb').Table('prod-linkedinCompanies').query(
#                 IndexName='LinkedinUsernameIndex',
#                 KeyConditionExpression=Key('linkedinUsername').eq(prospectsTable['experiences'][i]['linkedinCompanyUsername'])
#             )['Items']
#             all_company_classifications.append(company_classification_const(linkedinCompanies[0]['category'].replace(' - ', ' ')))
#         except:
#             try:
#                 linkedinCompanies = boto3.resource('dynamodb').Table('prod-linkedinCompanies').query(
#                     IndexName='LinkedinIdIndex',
#                     KeyConditionExpression=Key('linkedinId').eq(int(prospectsTable['experiences'][i]['linkedinCompanyId']))
#                 )['Items']
#                 all_company_classifications.append(company_classification_const(linkedinCompanies[0]['category'].replace(' - ', ' ')))
#             except:
#                 all_company_classifications.append('Missing')
            
#         if i == 0:
#             if prospectsTable['experiences'][i]['to'] == None:
#                 total_experience_months.append((int((dt.now() - dt.strptime(prospectsTable['experiences'][i]['from'][:10], "%Y-%m-%d")).days/30)))
#             else:
#                 total_experience_months.append((int((dt.strptime(prospectsTable['experiences'][i]['to'][:10], "%Y-%m-%d") - dt.strptime(prospectsTable['experiences'][i]['from'][:10], "%Y-%m-%d")).days/30)))
#             #precisa colocar o copy porque a variavel total_experience sera alterada, mas a last_experience nao pode mais ser alterada
#             last_experience_duration_months = total_experience_months.copy()
#             try:
#                 last_experience_descriptions = prospectsTable['experiences'][i]['description']
#             except:
#                 last_experience_descriptions = ''            
#         else:
#             if prospectsTable['experiences'][i]['to'] == None:
#                 total_experience_months.append((int((dt.now() - dt.strptime(prospectsTable['experiences'][i]['from'][:10], "%Y-%m-%d")).days/30)))
#                 total_experience_months_clean.append((int((dt.now() - dt.strptime(prospectsTable['experiences'][i]['from'][:10], "%Y-%m-%d")).days/30)))
#             else:
#                 total_experience_months.append((int((dt.strptime(prospectsTable['experiences'][i]['to'][:10], "%Y-%m-%d") - dt.strptime(prospectsTable['experiences'][i]['from'][:10], "%Y-%m-%d")).days/30)))
#                 total_experience_months_clean.append((int((dt.strptime(prospectsTable['experiences'][i]['to'][:10], "%Y-%m-%d") - dt.strptime(prospectsTable['experiences'][i]['from'][:10], "%Y-%m-%d")).days/30)))
#     except:
#         last_experience_duration_months=[0]
#         total_experience_months=[0]
#         last_experience_descriptions = ''
#         all_company_classifications = ['Missing']

# dict_prospects['prospect_companies_qty'] = len(set(prospect_companies))
# dict_prospects['total_experience_months'] = sum(total_experience_months)
# dict_prospects['experience_duration_months_min'] = min(total_experience_months)
# dict_prospects['experience_duration_months_clean_avg'] = np.mean((total_experience_months_clean))
# dict_prospects['experience_duration_months_clean_stddev'] =  np.std(total_experience_months_clean, ddof = 1)
# dict_prospects['last_experience_duration_months_to_avg'] = float(last_experience_duration_months[0]) - float(dict_prospects['experience_duration_months_clean_avg'])
# dict_prospects['last_experience_descriptions_word_count'] = float(len(last_experience_descriptions.split()))
# dict_prospects['last_company_classification'] = all_company_classifications[0]
# dict_prospects['all_company_classifications_word_count'] = float(len('; '.join(map(str, all_company_classifications)).replace("||", ",").split()))
dict_prospects

{'prospect_companies_qty': 5,
 'total_experience_months': 80,
 'experience_duration_months_min': 3,
 'experience_duration_months_clean_avg': 15.4,
 'experience_duration_months_clean_stddev': 12.973048986263793,
 'last_experience_duration_months_to_avg': -12.4,
 'last_experience_descriptions_word_count': 8.0,
 'last_company_classification': 'Missing',
 'all_company_classifications_word_count': 12.0,
 'declared_seniority': 'Mid-level',
 'declared_seniority_migration': 'Mid-level-to-Specialist',
 'prospect_location_state': 'São Paulo',
 'prospect_area_migration': 'Data-to-Agile',
 'prospect_smart_skills_qty': 23.0,
 'prospect_experiences_qty': 6.0}

In [38]:
# 'prospect_linkedin_about_word_count'
try:
    dict_prospects['prospect_linkedin_about_word_count'] = float(len(prospectsTable['linkedinAboutText'].split()))
except:
    dict_prospects['prospect_linkedin_about_word_count'] = 0
dict_prospects

{'prospect_companies_qty': 5,
 'total_experience_months': 80,
 'experience_duration_months_min': 3,
 'experience_duration_months_clean_avg': 15.4,
 'experience_duration_months_clean_stddev': 12.973048986263793,
 'last_experience_duration_months_to_avg': -12.4,
 'last_experience_descriptions_word_count': 8.0,
 'last_company_classification': 'Missing',
 'all_company_classifications_word_count': 12.0,
 'declared_seniority': 'Mid-level',
 'declared_seniority_migration': 'Mid-level-to-Specialist',
 'prospect_location_state': 'São Paulo',
 'prospect_area_migration': 'Data-to-Agile',
 'prospect_smart_skills_qty': 23.0,
 'prospect_experiences_qty': 6.0,
 'prospect_linkedin_about_word_count': 0}

In [39]:
# company_classification_migration
dict_prospects['company_classification_migration']=(dict_prospects['last_company_classification']+'-to-'+dict_jobs['job_company_classification']).strip()
dict_prospects

{'prospect_companies_qty': 5,
 'total_experience_months': 80,
 'experience_duration_months_min': 3,
 'experience_duration_months_clean_avg': 15.4,
 'experience_duration_months_clean_stddev': 12.973048986263793,
 'last_experience_duration_months_to_avg': -12.4,
 'last_experience_descriptions_word_count': 8.0,
 'last_company_classification': 'Missing',
 'all_company_classifications_word_count': 12.0,
 'declared_seniority': 'Mid-level',
 'declared_seniority_migration': 'Mid-level-to-Specialist',
 'prospect_location_state': 'São Paulo',
 'prospect_area_migration': 'Data-to-Agile',
 'prospect_smart_skills_qty': 23.0,
 'prospect_experiences_qty': 6.0,
 'prospect_linkedin_about_word_count': 0,
 'company_classification_migration': 'Missing-to-Empresa Tradicional B'}

In [40]:
del dict_jobs['job_seniority']
del dict_jobs['job_company_classification']

In [41]:
df = pd.concat([pd.DataFrame(dict_jobs, index=[0]), pd.DataFrame(dict_prospects, index=[0])], axis=1)
predict=h2o.mojo_predict_pandas(df, PathModelMojo).loc[:,('predict','p1')]
predict

There were 1 missing columns found in the input data set: {all_company_classifications_count}
Detected 2 unused columns in the input data set: {,all_company_classifications_word_count}


,predict,p1
0,1,0.148324


In [42]:
def criar_ratings(p1):
    if p1 <= 0.02940213015149787:
        return 1
    elif p1 <= 0.04974699465375872:
        return 2
    elif p1 <= 0.060006032316420675:
        return 3
    elif p1 <= 0.10576355645060076:
        return 4
    elif p1 <= 0.11898587992365063:
        return 5
    elif p1 <= 0.13668278644593498:
        return 6
    elif p1 <= 0.16319113762684134:
        return 7
    elif p1 <= 0.2150609643746747:
        return 8
    elif p1 <= 0.3666322315432273:
        return 9
    else:
        return 10

def criar_suggestion(predict):
    if predict == 0:
        return 'Repensar'
    elif predict == 1:
        return 'Abordar'
    else:
        return 'SUGGESTION_ERROR'

predict['suggestion'] = predict.apply(lambda x: criar_suggestion(x['predict']),axis=1).astype(str)

predict['rating'] = predict.apply(lambda x: criar_ratings(x['p1']),axis=1).astype(str)

predict

,predict,p1,suggestion,rating
0,1,0.148324,Abordar,7


In [43]:
df[['all_company_classifications_word_count',
'company_classification_migration',
'declared_seniority',
'declared_seniority_migration',
'experience_duration_months_clean_avg',
'experience_duration_months_clean_stddev',
'experience_duration_months_min',
'import_policy_word_count',
'job_area',
'job_technical_requirements_word_count',
'job_validation_questions_word_count',
'last_company_classification',
'last_experience_descriptions_word_count',
'last_experience_duration_months_to_avg',
'max_salary_offered',
'prospect_area_migration',
'prospect_companies_qty',
'prospect_experiences_qty',
'prospect_linkedin_about_word_count',
'prospect_location_state',
'prospect_smart_skills_qty',
'total_experience_months',
]].to_dict()

{'all_company_classifications_word_count': {0: 12.0},
 'company_classification_migration': {0: 'Missing-to-Empresa Tradicional B'},
 'declared_seniority': {0: 'Mid-level'},
 'declared_seniority_migration': {0: 'Mid-level-to-Specialist'},
 'experience_duration_months_clean_avg': {0: 15.4},
 'experience_duration_months_clean_stddev': {0: 12.973048986263793},
 'experience_duration_months_min': {0: 3},
 'import_policy_word_count': {0: 21.0},
 'job_area': {0: 'Agile'},
 'job_technical_requirements_word_count': {0: 135.0},
 'job_validation_questions_word_count': {0: 126.0},
 'last_company_classification': {0: 'Missing'},
 'last_experience_descriptions_word_count': {0: 8.0},
 'last_experience_duration_months_to_avg': {0: -12.4},
 'max_salary_offered': {0: 14000.0},
 'prospect_area_migration': {0: 'Data-to-Agile'},
 'prospect_companies_qty': {0: 5},
 'prospect_experiences_qty': {0: 6.0},
 'prospect_linkedin_about_word_count': {0: 0},
 'prospect_location_state': {0: 'São Paulo'},
 'prospect_sma

# 4. Create Lambda Handler to Deploy with API Gateway

In [48]:
def lambda_handler(event, context):
    import h2o
    import pandas as pd
    import numpy as np
    import json
    import boto3
    from boto3.dynamodb.conditions import Key
    import datetime as dt
    import queue
    from threading import Thread
    from decimal import Decimal
    import re
    
    #Best Model ID:
    BestModelId='GBM_grid_1_AutoML_1_20220602_224459_model_101.zip'
    
    #Keep the ratings ranges updated
    def ratings(p1):
        if p1 <= 0.0382443443235362:
            return 1
        if p1 <= 0.0530430015585711:
            return 2
        if p1 <= 0.0673746428820707:
            return 3
        if p1 <= 0.0784943959938068:
            return 4
        if p1 <= 0.0995685833288207:
            return 5
        if p1 <= 0.1190589979996429:
            return 6
        if p1 <= 0.1493789514138822:
            return 7
        elif p1 <= 0.1744927522205538:
            return 8
        elif p1 <= 0.2207991656421561:
            return 9
        else:
            return 10
            
    def company_classification_const(s):
        if s == 'Empresa pequena':
            out='Empresa pequena'
        elif s == 'Empresas pequenas':
            out='Empresa pequena'
        elif s == 'Empresa Tradicional A':
            out='Empresa Tradicional A'
        elif s == 'Empresa Tradicional B':
            out='Empresa Tradicional B'
        elif s == 'Empresa Tradicional C':
            out='Empresa Tradicional C'
        elif s == 'High Tech A':
            out='High Tech A'
        elif s == 'High Tech B':
            out='High Tech B'
        elif s == 'High Tech C':
            out='High Tech C'
        elif s == 'High Tech D':
            out='High Tech D'
        elif s == 'Software House A':
            out='Software House A'
        elif s == 'Software House B':
            out='Software House B'
        elif s == 'Software House C':
            out='Software House C'
        elif s == 'not_mapped':
            out='Missing'
        elif s in ('', 'null', None, 'None'):
            out='Missing'
        else:
            out='UNKNOWN'
        return out
            
    linkedinUsername = event['queryStringParameters']['linkedinUsername']
    jobId = event['queryStringParameters']['jobId']
    
    #Eh necessario criar duas boto3 para trabalhar com multiple threads. Uma para cada thread
    
    #boto3_Session para a tabela jobs
    boto_jobs = boto3.Session(region_name='us-east-1')
    
    #boto3_Session para a tabela prospects
    boto_prospects = boto3.Session(region_name='us-east-1')
    
#     #boto3_Session para a tabela jobs
#     boto_jobs = boto3.Session(region_name='us-east-1',
#         aws_access_key_id='xxxxxxxxx',
#         aws_secret_access_key='xxxxxxxx')

#     #boto3_Session para a tabela prospects
#     boto_prospects = boto3.Session(region_name='us-east-1',
#         aws_access_key_id='xxxxxxxxx',
#         aws_secret_access_key='xxxxxxxxx')
    
    #Criar funcoes de consulta ao DynamoDB para serem executadas em paralelo diminuindo o tempo
    def dynamodb_jobs_clients_companies(jobId, boto3):        
        jobsTable = boto3.resource('dynamodb').Table('prod-jobsTable').query(
            IndexName='GSI1',
            KeyConditionExpression=Key('backofficeId').eq(int(jobId))
        )['Items']
    
        clientsTable = boto3.resource('dynamodb').Table('prod-clientsTable').query(
            IndexName='idBackofficeIndex',
            KeyConditionExpression = (Key('source').eq('backoffice') & Key('idBackoffice').eq(int(jobsTable[0]['companyId'])))
        )['Items']
        
        if clientsTable[0]['linkedinURL'][-1] == '/':
            linkedin_company_user_name = clientsTable[0]['linkedinURL'].split('/')[-2]
        else:
            linkedin_company_user_name = clientsTable[0]['linkedinURL'].split('/')[-1]
    
        linkedinCompanies = boto3.resource('dynamodb').Table('prod-linkedinCompanies').query(
            IndexName='LinkedinUsernameIndex',
            KeyConditionExpression=Key('linkedinUsername').eq(linkedin_company_user_name)
        )['Items']
        
        return_object = {} 
        return_object['jobsTable'] = jobsTable
        return_object['linkedinCompanies'] = linkedinCompanies
        return return_object
        
    def dynamodb_prospects_companies(linkedinUsername, boto3):        
        prospectsTable = boto3.resource('dynamodb').Table('prod-prospectsTable').get_item(Key={'linkedinUsername': linkedinUsername})['Item']
    
        dict_prospects={}
        total_experience_months=[]
        total_experience_months_clean=[]
        prospect_companies=[]        
        all_company_classifications=[]
    def dynamodb_prospects_companies(linkedinUsername, boto3):        
        prospectsTable = boto3.resource('dynamodb').Table('prod-prospectsTable').get_item(Key={'linkedinUsername': linkedinUsername})['Item']
    
        dict_prospects={}
        total_experience_months=[]
        total_experience_months_clean=[]
        prospect_companies=[]        
        all_company_classifications=[]
        try:
            for i in range(len(prospectsTable['experiences'])):
                prospect_companies.append(prospectsTable['experiences'][i]['linkedinCompanyName'])
    
                try:
                    linkedinCompanies = boto3.resource('dynamodb').Table('prod-linkedinCompanies').query(
                        IndexName='LinkedinUsernameIndex',
                        KeyConditionExpression=Key('linkedinUsername').eq(prospectsTable['experiences'][i]['linkedinCompanyUsername'])
                    )['Items']
                    all_company_classifications.append(company_classification_const(linkedinCompanies[0]['category'].replace(' - ', ' ')))
                except Exception as e:                    
                    print("e.1: "+ str(e))
                    try:
                        linkedinCompanies = boto3.resource('dynamodb').Table('prod-linkedinCompanies').query(
                            IndexName='LinkedinIdIndex',
                            KeyConditionExpression=Key('linkedinId').eq(int(prospectsTable['experiences'][i]['linkedinCompanyId']))
                        )['Items']
                        all_company_classifications.append(company_classification_const(linkedinCompanies[0]['category'].replace(' - ', ' ')))
                    except Exception as e:
                        print("e.2: "+ str(e))
                        all_company_classifications.append('Missing')
    
                if i == 0:
                    if prospectsTable['experiences'][i]['to'] == None:
                        total_experience_months.append((int((dt.datetime.now() - dt.datetime.strptime(prospectsTable['experiences'][i]['from'][:10], "%Y-%m-%d")).days/30)))
                    else:
                        total_experience_months.append((int((dt.datetime.strptime(prospectsTable['experiences'][i]['to'][:10], "%Y-%m-%d") - dt.datetime.strptime(prospectsTable['experiences'][i]['from'][:10], "%Y-%m-%d")).days/30)))
                    #precisa colocar o copy porque a variavel total_experience sera alterada, mas a last_experience nao pode mais ser alterada
                    last_experience_duration_months = total_experience_months.copy()
                    try:
                        last_experience_descriptions = prospectsTable['experiences'][i]['description']
                    except Exception as e:
                        print("e.3: "+ str(e))
                        last_experience_descriptions = ''
                else:
                    try:
                        if prospectsTable['experiences'][i]['to'] == None:
                            total_experience_months.append((int((dt.datetime.now() - dt.datetime.strptime(prospectsTable['experiences'][i]['from'][:10], "%Y-%m-%d")).days/30)))
                            total_experience_months_clean.append((int((dt.datetime.now() - dt.datetime.strptime(prospectsTable['experiences'][i]['from'][:10], "%Y-%m-%d")).days/30)))
                        else:
                            total_experience_months.append((int((dt.datetime.strptime(prospectsTable['experiences'][i]['to'][:10], "%Y-%m-%d") - dt.datetime.strptime(prospectsTable['experiences'][i]['from'][:10], "%Y-%m-%d")).days/30)))
                            total_experience_months_clean.append((int((dt.datetime.strptime(prospectsTable['experiences'][i]['to'][:10], "%Y-%m-%d") - dt.datetime.strptime(prospectsTable['experiences'][i]['from'][:10], "%Y-%m-%d")).days/30)))
                    except Exception as e:
                        print("e.3b: "+ str(e))
        except Exception as e:
            print("e.4: "+ str(e))
            last_experience_duration_months=[0]
            total_experience_months_clean=[0,0.1]
            total_experience_months=[0]
            last_experience_descriptions = ''
            all_company_classifications = ['Missing']
    
        dict_prospects['prospect_companies_qty'] = len(set(prospect_companies))
        dict_prospects['total_experience_months'] = sum(total_experience_months)
        dict_prospects['experience_duration_months_min'] = min(total_experience_months)
        dict_prospects['experience_duration_months_clean_avg'] = np.nan_to_num(np.mean((total_experience_months_clean)))
        dict_prospects['experience_duration_months_clean_stddev'] =  np.nan_to_num(np.std(total_experience_months_clean, ddof = 1))
        dict_prospects['last_experience_duration_months_to_avg'] = float(last_experience_duration_months[0]) - float(dict_prospects['experience_duration_months_clean_avg'])
        dict_prospects['last_experience_descriptions_word_count'] = float(len(last_experience_descriptions.split()))
        dict_prospects['last_company_classification'] = all_company_classifications[0]
#         dict_prospects['all_company_classifications_word_count'] = float(len('; '.join(map(str, all_company_classifications)).replace("||", ",").split()))
        dict_prospects['all_company_classifications_count'] = (sum(map((lambda x: (0 if x.strip() == 'Missing' else 1)), all_company_classifications)))
        
        return_object = {} 
        return_object['prospectsTable'] = prospectsTable
        return_object['dict_prospects'] = dict_prospects
        return return_object

    #Executar as duas funcoes de forma assincrona
    try:
        #criar q1 e q2 para receber o return de cada funcao
        q1 = queue.Queue()
        q2 = queue.Queue()
        #criar as duas tarefas
        t1 = Thread(target = lambda q, arg1, arg2 : q.put(dynamodb_jobs_clients_companies(arg1, arg2)), args = (q1, jobId, boto_jobs))
        t2 = Thread(target = lambda q, arg1, arg2 : q.put(dynamodb_prospects_companies(arg1, arg2)), args = (q2, linkedinUsername, boto_prospects))
        #iniciar as duas tarefas simutaneamente.
        t1.start()
        t2.start()
        #unir as duas tarefas para garantir que a proxima tarefa seja iniciada somente quando as duas terminarem
        t1.join()
        t2.join()
        
        #recebar o return da funcao
        while not q1.empty():
            return_object_jobs_clients_companies = q1.get()
        while not q2.empty():
            return_object_prospects_companies = q2.get()
    except Exception as e:
        print("e.5: "+ str(e))
        #criar q1 e q2 para receber o return de cada funcao
        q1 = queue.Queue()
        q2 = queue.Queue()
        #criar as duas tarefas
        t1 = Thread(target = lambda q, arg1, arg2 : q.put(dynamodb_jobs_clients_companies(arg1, arg2)), args = (q1, jobId, boto3))
        t2 = Thread(target = lambda q, arg1, arg2 : q.put(dynamodb_prospects_companies(arg1, arg2)), args = (q2, linkedinUsername, boto3))
        #iniciar as duas tarefas simutaneamente.
        t1.start()
        t2.start()
        #unir as duas tarefas para garantir que a proxima tarefa seja iniciada somente quando as duas terminarem
        t1.join()
        t2.join()
        
        #recebar o return da funcao
        while not q1.empty():
            return_object_jobs_clients_companies = q1.get()
        while not q2.empty():
            return_object_prospects_companies = q2.get()
            
    jobsTable = return_object_jobs_clients_companies['jobsTable']
    linkedinCompanies = return_object_jobs_clients_companies['linkedinCompanies']
    prospectsTable = return_object_prospects_companies['prospectsTable']
    dict_prospects = return_object_prospects_companies['dict_prospects']

    #Start empty dictionary
    dict_jobs={}

    ## job_seniority = 'profile.seniority'
    def job_seniority_const(s):
        if s in ("Júnior", "junior", "Junior"):
            out="Junior"
        elif s in ("Pleno", "pleno"):
            out="Mid-level"
        elif s in ("Senior", "Sênior", "senior"):
            out="Senior"
        elif s in ("Especialista", "especialista"):
            out="Specialist"
        elif s == "Tech Lead":
            out="Tech Lead"
        elif s == "Tech Manager":
            out="Tech Manager"
        elif s in ('', 'null', None, 'None'):
            out="Missing"
        else:
            out="UNKNOWN"
        return out
    
    dict_jobs['job_seniority'] = job_seniority_const(jobsTable[0]['profile']['seniority'])

    ## job_area = 'area'
    try:
        dict_jobs['job_area'] = jobsTable[0]['area']
    except Exception as e:
        print("e.6: "+ str(e))
        dict_jobs['job_area'] = 'Missing'

    ## max_salary_offered = 'maxsalary'
    try:
        dict_jobs['max_salary_offered'] = float(jobsTable[0]['maxSalary'])
    except Exception as e:
        print("e.7: "+ str(e))
        dict_jobs['max_salary_offered'] = 0
        
    ## import_policy_word_count = 'importPolicy'
    try:
        dict_jobs['import_policy_word_count'] = float(len(jobsTable[0]['importPolicy'].split()))
    except Exception as e:
        print("e.8: "+ str(e))
        dict_jobs['import_policy_word_count'] = 0

    ## job_technical_requirements_word_count = 'alignments.intendedTechnicalInfo'
    try:
        dict_jobs['job_technical_requirements_word_count'] = float(len(jobsTable[0]['alignments']['intendedTechnicalInfo'].split()))
    except Exception as e:
        print("e.9: "+ str(e))
        dict_jobs['job_technical_requirements_word_count'] = 0

    ## job_validation_questions_word_count = 'validationQuestions'
    try:
        dict_jobs['job_validation_questions_word_count'] = float(len(jobsTable[0]['validationQuestions'].split()))
    except Exception as e:
        print("e.10: "+ str(e))
        dict_jobs['job_validation_questions_word_count'] = 0
        
    ## company_classification
    try:
        dict_jobs['job_company_classification'] = company_classification_const(linkedinCompanies[0]['category'].replace(' - ', ' '))
    except Exception as e:
        print("e.11: "+ str(e))
        dict_jobs['job_company_classification'] = 'Missing'

    
    ## prospectsTable

    ## declared_seniority = 'declaredseniority'
    try:
        dict_prospects['declared_seniority'] = job_seniority_const(prospectsTable['declaredSeniority'])
    except Exception as e:
        print("e.12: "+ str(e))
        dict_prospects['declared_seniority'] = 'Missing'

    # 'declared_seniority_migration'
    dict_prospects['declared_seniority_migration'] = (dict_prospects['declared_seniority']+'-to-'+dict_jobs['job_seniority']).strip()

    #Criacao das variaveis de pais com base na variavel propesct_location do linkedin. Para comparar abordagens de talentos que moram fora do Brasil
    def prospect_country(country):
        if pd.isna(country):
            return "Missing"
        elif country == 'São Paulo':
            return 'Brazil'
        elif country == 'Rio de Janeiro':
            return 'Brazil'
        elif country == 'Campinas':
            return 'Brazil'
        elif country == 'Belo Horizonte':
            return 'Brazil'
        elif country == 'Porto Alegre':
            return 'Brazil'
        elif country == 'Curitiba':
            return 'Brazil'
        elif country == 'Brasília':
            return 'Brazil'
        elif country == 'Florianópolis':
            return 'Brazil'
        elif country == 'Salvador':
            return 'Brazil'
        elif country == 'Fortaleza':
            return 'Brazil'
        elif country == 'Recife':
            return 'Brazil'
        elif country == 'Manaus':
            return 'Brazil'
        elif country == 'Ribeirão Preto':
            return 'Brazil'
        elif country == 'Goiânia':
            return 'Brazil'
        elif country == 'João Pessoa':
            return 'Brazil'
        elif country == 'Londrina':
            return 'Brazil'
        elif country == 'Vitória':
            return 'Brazil'
        elif country == 'Cuiabá':
            return 'Brazil'
        elif country == 'Greater São Paulo Area':
            return 'Brazil'
        elif country == 'Natal':
            return 'Brazil'
        elif country == 'São luis':
            return 'Brazil'
        elif country == 'Brazil':
            return 'Brazil'
        elif country == 'Brasil':
            return 'Brazil'
        else:
            return 'Others'
        
    def prospect_region_international(region, country):
        if country not in ('Brazil', 'Missing'):
            return "International"
        else:
            return region
    try:
        dict_prospects['prospect_location_state'] = prospect_region_international(prospectsTable['location'].split(',')[-2].replace(" e Região", "").strip(), prospect_country(prospectsTable['location'].split(',')[-1].replace(" e Região", "").strip()))
    except Exception as e:
        print("e.13: "+ str(e))
        dict_prospects['prospect_location_state'] = 'Missing'
        
    ## prospect_area_migration
    try:    
        dict_prospects['prospect_area_migration'] = (prospectsTable['area'].split()[0]+'-to-'+dict_jobs['job_area']).strip()
    except Exception as e:
        print("e.14: "+ str(e))
        dict_prospects['prospect_area_migration'] = ('Missing'+'-to-'+dict_jobs['job_area']).strip()

    ## prospect_smart_skills_qty
    try:
        dict_prospects['prospect_smart_skills_qty'] = float(len(prospectsTable['smartTags']))
    except Exception as e:
        print("e.15: "+ str(e))
        dict_prospects['prospect_smart_skills_qty'] = 0.0

    ## prospect_experiences_qty
    try:
        dict_prospects['prospect_experiences_qty'] = float(len(prospectsTable['experiences']))
    except Exception as e:
        print("e.16: "+ str(e))
        dict_prospects['prospect_experiences_qty'] = 0.0
    
    # 'prospect_linkedin_about_word_count'
    try:
        dict_prospects['prospect_linkedin_about_word_count'] = float(len(prospectsTable['linkedinAboutText'].split()))
    except Exception as e:
        print("e.17: "+ str(e))
        dict_prospects['prospect_linkedin_about_word_count'] = 0
        
    # company_classification_migration
    dict_prospects['company_classification_migration']=(dict_prospects['last_company_classification']+'-to-'+dict_jobs['job_company_classification']).strip()
    
    del dict_jobs['job_seniority']
    del dict_jobs['job_company_classification']
    
    df = pd.concat([pd.DataFrame(dict_jobs, index=[0]), pd.DataFrame(dict_prospects, index=[0])], axis=1)
    
    predict=h2o.mojo_predict_pandas(df.set_index('last_company_classification', inplace=False), mojo_zip_path=BestModelId, genmodel_jar_path='h2o-genmodel.jar', verbose=False).loc[:,('predict','p1')]
            
    def suggestion(predict):
        if predict == 0:
            return 'Repensar'
        elif predict == 1:
            return 'Abordar'
        else:
            return 'SUGGESTION_ERROR'
    
    predict['suggestion'] = predict.apply(lambda x: suggestion(x['predict']),axis=1).astype(str)
    
    predict['rating'] = predict.apply(lambda x: ratings(x['p1']),axis=1).astype(str)

    body = {
        "message": "Prediction executed successfully!"        
    }

    body['probability'] = round(predict['p1'][0],4)
    body['rating'] = predict['rating'][0]
    body['suggestion'] = predict['suggestion'][0]
    
    response = {
        "statusCode": 200,
        "body": json.dumps(body),
        "headers": {
            "Access-Control-Allow-Origin": "*"
        }
    }
    
    return response

# # DEV
# event={
#     "queryStringParameters":{"jobId":"2329",
#                              "linkedinUsername":"patriciakano"}
# }
# context='context'
# print(lambda_handler(event, context))

# 5. Test Lambda + API Gateway

In [44]:
dict={
  "jobId": 1198,
  "linkedinUsername": "maxuel-reis-1b343621"
}

In [45]:
%%timeit
import requests

# Parâmetros do modelo
endpoint_modelo = 'https://lp1027iq13.execute-api.us-east-1.amazonaws.com/dev/get-predict'
# https://e87iurnmfh.execute-api.us-east-1.amazonaws.com/dev/get-predict

# Definição dos inputs
params = dict

resultado = requests.get(endpoint_modelo, params = params)
print(resultado.json())

{'message': 'Prediction executed successfully!', 'probability': 0.102, 'rating': '6', 'suggestion': 'Repensar', 'image_path': '1198_maxuel-reis-1b343621_20220702200510489698.png'}
{'message': 'Prediction executed successfully!', 'probability': 0.102, 'rating': '6', 'suggestion': 'Repensar', 'image_path': '1198_maxuel-reis-1b343621_20220702200513972178.png'}
{'message': 'Prediction executed successfully!', 'probability': 0.102, 'rating': '6', 'suggestion': 'Repensar', 'image_path': '1198_maxuel-reis-1b343621_20220702200516973655.png'}
{'message': 'Prediction executed successfully!', 'probability': 0.102, 'rating': '6', 'suggestion': 'Repensar', 'image_path': '1198_maxuel-reis-1b343621_20220702200520137308.png'}
{'message': 'Prediction executed successfully!', 'probability': 0.102, 'rating': '6', 'suggestion': 'Repensar', 'image_path': '1198_maxuel-reis-1b343621_20220702200523370732.png'}
{'message': 'Prediction executed successfully!', 'probability': 0.102, 'rating': '6', 'suggestion': 

In [47]:
# df_end_point = datascored_df.query('job_id == 2259')[['linkedin_user_name', 'approach_created_at_date', 'job_id', 'predict', 'p0', 'p1']].reset_index()

# for i in range(len(df_end_point)):
#     temp = df_end_point.loc[i:i, ('linkedin_user_name','job_id', 'p1')].to_dict()

#     dict={
#       "jobId": temp['job_id'][i],
#       "linkedinUsername": temp['linkedin_user_name'][i]
#     }  

#     # Parâmetros do modelo
#     endpoint_modelo = 'https://io9fmel6yc.execute-api.us-east-1.amazonaws.com/dev/get-predict'
#     # https://e87iurnmfh.execute-api.us-east-1.amazonaws.com/dev/get-predict

#     # Definição dos inputs
#     params = dict

#     resultado = requests.get(endpoint_modelo, params = params)
#     temp2 = resultado.json()
#     print(round(temp['p1'][i],4))
#     print(temp2['probability'])
    
#     print(round(temp['p1'][i],4) == temp2['probability'])